## 월별 VOD 시청기록

<br><hr>

### 00. 기본 설정

In [4]:
import pandas as pd
import zipfile

# 경고 메시지 출력 X
import warnings
warnings.filterwarnings("ignore")

<br><hr>

### 01. 데이터 불러오기

In [6]:
dataframes = []

for month in range(1, 12):  # 1부터 11까지 반복
    file_path = f"../data/vods/2023{str(month).zfill(2)}_VOD.zip"  # 파일 경로 생성
    try:
        # ZIP 파일 내 CSV 읽기
        df = pd.read_csv(file_path, compression='zip')  # ZIP 파일 내 CSV 읽기
        dataframes.append(df)  # 데이터프레임 리스트에 추가
    except Exception as e:
        print(f"Error reading file {file_path}: {e}")

<br><hr>

### 02. 1월~11월 영화 시청기록 병합

In [7]:
# shape 확인
for i in range(11):
  print(dataframes[i].shape)

(6325398, 9)
(7350110, 9)
(7662099, 9)
(1971241, 9)
(7712374, 9)
(7165520, 9)
(8004336, 9)
(7665944, 9)
(6919315, 9)
(1178159, 9)
(6416074, 9)


In [13]:
dataframes_movies = {}

for idx, df in enumerate(dataframes, start=1):
    month = str(idx).zfill(2)
    var_name = f"m{month}_movies"
    
    # 'CT_CL' 값이 '영화'인 행 필터링
    df_movies = df[df['CT_CL'] == '영화']
    dataframes_movies[var_name] = df_movies

    # shape 확인
    print(f"{month}월: {dataframes_movies[var_name].shape}")

01월: (1257149, 9)
02월: (1593025, 9)
03월: (1878906, 9)
04월: (463937, 9)
05월: (1775975, 9)
06월: (1849426, 9)
07월: (1890441, 9)
08월: (1579962, 9)
09월: (1373404, 9)
10월: (250182, 9)
11월: (1422513, 9)


In [14]:
# 데이터프레임 병합
combined_movies = pd.concat(dataframes_movies.values(), ignore_index=True)

# 결과 출력
print(combined_movies.shape)

(15334920, 9)


<br><hr>

### 03. 병합 데이터 전처리

- 장르 삭제 ('성인', '에로틱', '영화해설')
- 사용자 hash id 변경

In [18]:
combined_movies.head(1)

,sha2_hash,asset,asset_nm,CT_CL,genre_of_ct_cl,use_tms,disp_rtm,strt_dt,category
0,ea62338ac5b6b11cf02ef8bf1889d1a063cec2c2493937...,cjc|M5140475LSGL08601501,압꾸정,영화,코미디,2.0,01:51,2.023012e+13,영화/(HD)극장동시상영관


In [19]:
# 장르 확인
combined_movies['genre_of_ct_cl'].value_counts()

genre_of_ct_cl
액션/어드벤쳐    3617254
성인         3446828
멜로         2605584
드라마        2120459
코미디         976739
공포/스릴러      636281
기타          630081
애니메이션       628804
SF/환타지      304701
무협          184352
다큐멘터리        93639
로맨틱코미디       41518
역사           26005
단편           18792
뮤지컬           2772
서부             957
에로틱            151
영화해설             3
Name: count, dtype: int64

In [26]:
# '성인', '에로틱', '영화해설' 삭제
movies_filtered = combined_movies[~combined_movies['genre_of_ct_cl'].isin(['성인', '에로틱', '영화해설'])]
# 인덱스 초기화
movies_filtered.reset_index(inplace=True, drop=True)
movies_filtered.shape    # 1533만 4920개 >> 1188만 7938개 (344만 6982개 삭제)

(11887938, 9)

In [27]:
# 전체 사용자 수 확인
movies_filtered["sha2_hash"].nunique()  # 57만 4019명

574019

In [29]:
# 사용자 hash id 다시 지정
unique_users = movies_filtered["sha2_hash"].unique()
user_mapping = {user_id: f"user{str(i+1).zfill(6)}" for i, user_id in enumerate(unique_users)}

# 사용자 ID를 매핑된 값으로 수정
movies_filtered["userId"] = movies_filtered["sha2_hash"].map(user_mapping)

##### *열 삭제 전 csv 저장*

In [ ]:
movies_filtered.to_csv('../data/movies_시청기록병합_열전체.csv', encoding='utf8', index=False)

<br><hr>

### 03. 병합 데이터 전처리 - 2

- 사용하지 않을 column 삭제 (`'sha2_hash', 'CT_CL', 'genre_of_ct_cl', 'category'`)

In [31]:
movies_filtered.columns

Index(['sha2_hash', 'asset', 'asset_nm', 'CT_CL', 'genre_of_ct_cl', 'use_tms',
       'disp_rtm', 'strt_dt', 'category', 'userId'],
      dtype='object')

In [35]:
movies_dropped = movies_filtered.drop(['sha2_hash', 'CT_CL', 'genre_of_ct_cl', 'category'], axis=1)
print(movies_dropped.shape)

(11887938, 6)


In [37]:
movies_dropped.to_csv('../data/movies_시청기록병합.csv', index=False, encoding='utf8')